In [1]:
import os

import pickle

import numpy as np
import pandas as pd

import sys
sys.path.append(r'../')

from utils.WilmanDB import *
from utils.spectrum import *
from utils.methods import skymodel_to_img
from utils.methods import check_mem

from utils.galaxy import RSgen

import matplotlib.pyplot as plt


In [2]:
import copy

import pickle


import utils.methods as mthd
from utils.functions import in_FoV
from datetime import datetime

from astropy import convolution
from astropy.io import fits as fits
import astropy.units as u
from radio_beam import Beam

from scipy.ndimage.interpolation import shift
from scipy.signal import fftconvolve

In [3]:
from skytools import GenSkyModel

In [4]:
class Simulate:
    def __init__(self, Params):
        self.Params = Params
        self.skymodel = []
        self.galaxies_simulated = []
        self.total_number_of_simulated_galaxies = 0
        self.n_rq = 0
        self.n_fr1 = 0
        self.n_fr2 = 0
        self.n_sf = 0
        self.n_sb = 0
        self.init_switch()
        self.rsg = RSgen(self.Params)
    
    def init_switch(self):
        """
        Initialize simulation switches.
            0/False: No
            1/Ture: Yes
        """
        self.rq_switch = 1 
        self.fr1_switch = 1  
        self.fr2_switch = 1  
        self.sf_switch = 1 
        self.sb_switch = 1
        if self.Params["number_of_rq"] == 0:
            self.rq_switch = False
        if self.Params["number_of_fr1"] == 0:
            self.fr1_switch = False
        if self.Params["number_of_fr2"] == 0:
            self.fr2_switch = False
        if self.Params["number_of_sf"] == 0:
            self.sf_switch = False
        if self.Params["number_of_sb"] == 0:
            self.sb_switch = False
        return 0
    
    def check_distance(self, x, y):
        """
        Check whether the distance between the new source and \
        other simulated sources is greater than dmin.
        Return: 
            True if the distance is greater than dmin, otherwise return False.
        """
        for galaxy in self.skymodel:
            distance = np.sqrt((galaxy[:, 0] - x) ** 2 + (galaxy[:, 1] - y) ** 2)
            if np.sum(distance < self.Params['dmin']) > 1:
                return False
            else:
                return True
        
    def make_skymodel(self, galaxies):
        for i in galaxies.index:
            if galaxies.loc[i]['type'] == 'RQQ':
                if self.rq_switch:
                    g = DB.rq.loc[DB.rq['galaxy'] == galaxies.loc[i]['galaxy']]
                    pixels = self.rsg.rq(g)
                else:
                    continue
            if galaxies.loc[i]['type'] == 'FR1':
                if self.fr1_switch:
                    g = DB.fr1.loc[DB.fr1['galaxy'] == galaxies.loc[i]['galaxy']]
                    pixels = self.rsg.fr1Wilman(g)
                else:
                    continue
            if galaxies.loc[i]['type'] == 'FR2':
                if self.fr2_switch:
                    g = DB.fr2.loc[DB.fr2['galaxy'] == galaxies.loc[i]['galaxy']]
                    pixels = self.rsg.fr2Wilman(g)
                else:
                    continue
            if galaxies.loc[i]['type'] == 'SF':
                if self.sf_switch:
                    g = DB.sf.loc[DB.sf['galaxy'] == galaxies.loc[i]['galaxy']]
                    pixels = self.rsg.sfsb(g)
                else:
                    continue
            if galaxies.loc[i]['type'] == 'SB':
                if self.sb_switch:
                    g = DB.sb.loc[DB.sb['galaxy'] == galaxies.loc[i]['galaxy']]
                    pixels = self.rsg.sfsb(g)
                else:
                    continue

            if galaxies.loc[i]['galaxy'] in set(self.galaxies_simulated):
                continue
            is_greater_than_dmin = True
            if len(self.galaxies_simulated) > 1 and self.Params['dmin'] > 0:
                for pix in pixels:
                    x = pix[0]
                    y = pix[1]
                    if not self.check_distance(x, y):
                        is_greater_than_dmin = False
                        continue           
                if is_greater_than_dmin:
                    self.skymodel.append(np.array(pixels))
                    self.galaxies_simulated.append(galaxies.loc[i]['galaxy'])
                    if galaxies.loc[i]['type'] == 'RQQ':
                        self.n_rq += 1
                    if galaxies.loc[i]['type'] == 'FR1':
                        self.n_fr1 += 1              
                    if galaxies.loc[i]['type'] == 'FR2':
                        self.n_fr2 += 1 
                    if galaxies.loc[i]['type'] == 'SF':
                        self.n_sf += 1
                    if galaxies.loc[i]['type'] == 'SB':
                        self.n_sb += 1
            else:
                self.skymodel.append(np.array(pixels))
                self.galaxies_simulated.append(galaxies.loc[i]['galaxy'])
                if galaxies.loc[i]['type'] == 'RQQ':
                    self.n_rq += 1
                if galaxies.loc[i]['type'] == 'FR1':
                    self.n_fr1 += 1              
                if galaxies.loc[i]['type'] == 'FR2':
                    self.n_fr2 += 1 
                if galaxies.loc[i]['type'] == 'SF':
                    self.n_sf += 1
                if galaxies.loc[i]['type'] == 'SB':
                    self.n_sb += 1
            self.total_number_of_simulated_galaxies = self.n_rq + self.n_fr1 + self.n_fr2 + self.n_sb + self.n_sf
            
            if self.n_rq >= self.Params["number_of_rq"]:
                self.rq_switch = False
            if self.n_fr1 >= self.Params["number_of_fr1"]:
                self.fr1_switch = False
            if self.n_fr2 >= self.Params["number_of_fr2"]:
                self.fr2_switch = False
            if self.n_sf >= self.Params["number_of_sf"]:
                self.sf_switch = False
            if self.n_sb >= self.Params["number_of_sb"]:
                self.sb_switch = False
            
            print("Total number of simulated galaxies: %d" %(self.total_number_of_simulated_galaxies))
            print("RQ: %d; FR1: %d; FR2: %d; SB: %d; SF: %d." %(self.n_rq, self.n_fr1, self.n_fr2, self.n_sb, self.n_sf))
        return self.skymodel


class Parameters:
    def __init__(self, PARAMS):
        self.PARAMS = PARAMS
        
    def get_image_size(self):
        return self.PARAMS['image_size']
    
    def get_fov_deg(self):
        return self.PARAMS['fov_deg']
    
    def get_fov_arcsec(self):
        return self.PARAMS['fov_arcsec']
    
    def get_pix_deg(self):
        return self.PARAMS['pix_size_deg']
    
    def get_pix_size(self):
        return self.PARAMS['pix_size_arcsec']
    
    def get_pix_area(self):
        return self.PARAMS['pix_area_arcsec2']
    
    def get_ra_min(self):
        return self.PARAMS['ra_min']
    
    def get_dec_min(self):
        return self.PARAMS['dec_min']
    
    def get_dmin(self):
        return self.PARAMS['dmin']
    
    def get_num_rq(self):
        return self.PARAMS['number_of_rq']
      
    def get_num_fr1(self):
        return self.PARAMS['number_of_fr1']
    
    def get_num_fr2(self):
        return self.PARAMS['number_of_fr2']
    
    def get_num_sf(self):
        return self.PARAMS['number_of_sf']
    
    def get_num_sb(self):
        return self.PARAMS['number_of_sb']
    
    def get_ref_freqs(self):
        return self.PARAMS['frequency']
    
    def get_min_flux(self):
        return self.PARAMS['minimum_flux']
    
    def get_max_flux(self):
        return self.PARAMS['maximum_flux']
    
    def get_sim_freqs(self):
        return self.PARAMS['simulated_freqs']

In [5]:
def read_skymodel(fn):
    pickle_open = open(fn, 'rb')
    skymodel_151 = pickle.load(pickle_open)
    pickle_open.close()
    return skymodel_151

def save_skymodel(fn, skymodel):
    pickle_skymodel = open( fn, 'wb' )
    pickle.dump( skymodel, pickle_skymodel )
    pickle_skymodel.close()

In [6]:
pix_arcsec = 1
fov_deg = 5
fov_arcsec = fov_deg * 3600.0
ref_freq = 158
simulated_freqs = np.linspace(158, 162, 51)

img_size = int(fov_arcsec / pix_arcsec)

pix_size_deg = fov_deg / img_size
pix_size_arcsec = fov_arcsec / img_size
pix_area_arcsec2 = pix_size_arcsec ** 2
ra_min = -1 * fov_deg / 2.0
dec_min = -1 * fov_deg / 2.0

Params = {
    "img_size": img_size,
    "fov_deg": fov_deg,
    "fov_arcsec": fov_arcsec,
    "pix_deg": pix_size_deg,
    "pix_size": pix_size_arcsec,
    "pix_area": pix_area_arcsec2,
    "ra_min": ra_min,
    "dec_min": dec_min,
    "dmin": 100, # The minimum distance between sources. Its unit is pixel.
    "number_of_rq": 200*3,
    "number_of_fr1": 95*3,
    "number_of_fr2": 5*3,
    "number_of_sf": 1200*3,
    "number_of_sb": 500*3,
    "frequency": 158,
    "minimum_flux": 1.7536435940177778e-07, # Jy
    "maximum_flux": 17000,
    "simulated_freqs": simulated_freqs
}
## If you don't want to simulate some kind of galaxy, just set the number of it to 0.
print("Field of View: %.2f deg\nPixel Size: %.2f arcsec\nImage Size: %d" %(fov_deg, pix_size_arcsec, img_size))
print("Simulated Frequencies:\n", simulated_freqs)
check_mem(img_size)

Sim = Simulate(Params)
DB = WilmanDB(dbpath='../data')

Field of View: 5.00 deg
Pixel Size: 1.00 arcsec
Image Size: 18000
Simulated Frequencies:
 [158.   158.08 158.16 158.24 158.32 158.4  158.48 158.56 158.64 158.72
 158.8  158.88 158.96 159.04 159.12 159.2  159.28 159.36 159.44 159.52
 159.6  159.68 159.76 159.84 159.92 160.   160.08 160.16 160.24 160.32
 160.4  160.48 160.56 160.64 160.72 160.8  160.88 160.96 161.04 161.12
 161.2  161.28 161.36 161.44 161.52 161.6  161.68 161.76 161.84 161.92
 162.  ]
Ava: 35.54 GB; Req: 2.59 GB
Loading data from ../data/wilmandb_rqq.h5.
Finish loading data
Loading data from ../data/wilmandb_fr1.h5.
Finish loading data
Loading data from ../data/wilmandb_fr2.h5.
Finish loading data
Loading data from ../data/wilmandb_sf.h5.
Finish loading data
Loading data from ../data/wilmandb_sb.h5.
Finish loading data


In [7]:
PR = Parameters(Params)

In [8]:
def print_Params( Params ):
    for key in Params:
        print(key, ' : ', Params[key])

In [9]:
freq = Params["frequency"]
flux_min = Params["minimum_flux"]
flux_max = Params["maximum_flux"]
#fov = [-1 * Params["fov_deg"]/2, Params["fov_deg"]/2]
fov = [-1 * 2.4, 2.4]
bins = 20

while Sim.n_fr2 < Params['number_of_fr2']:
    galaxies = DB.sampling2(Params['number_of_fr2'], 'FR2', freq, flux_min, flux_max, fov)
    Sim.make_skymodel(galaxies)
while Sim.n_fr1 < Params['number_of_fr1']:
    galaxies = DB.sampling2(Params['number_of_fr1'], 'FR1', freq, flux_min, flux_max, fov)
    Sim.make_skymodel(galaxies)
while Sim.n_rq < Params['number_of_rq']:
    galaxies = DB.sampling2(Params['number_of_rq'], 'RQ', freq, flux_min, flux_max, fov)
    Sim.make_skymodel(galaxies)
while Sim.n_sf < Params['number_of_sf']:
    galaxies = DB.sampling2(Params['number_of_sf'], 'SF', freq, flux_min, flux_max, fov)
    Sim.make_skymodel(galaxies)
while Sim.n_sb < Params['number_of_sb']:
    galaxies = DB.sampling2(Params['number_of_sb'], 'SB', freq, flux_min, flux_max, fov)
    Sim.make_skymodel(galaxies)

Minimum flux: 1.45e-02; Maximum flux: 9.14e+00.
Total number of simulated galaxies: 1
RQ: 0; FR1: 0; FR2: 1; SB: 0; SF: 0.
Total number of simulated galaxies: 2
RQ: 0; FR1: 0; FR2: 2; SB: 0; SF: 0.
Total number of simulated galaxies: 3
RQ: 0; FR1: 0; FR2: 3; SB: 0; SF: 0.
Total number of simulated galaxies: 4
RQ: 0; FR1: 0; FR2: 4; SB: 0; SF: 0.
Total number of simulated galaxies: 5
RQ: 0; FR1: 0; FR2: 5; SB: 0; SF: 0.
Total number of simulated galaxies: 6
RQ: 0; FR1: 0; FR2: 6; SB: 0; SF: 0.
Total number of simulated galaxies: 7
RQ: 0; FR1: 0; FR2: 7; SB: 0; SF: 0.
Total number of simulated galaxies: 8
RQ: 0; FR1: 0; FR2: 8; SB: 0; SF: 0.
Total number of simulated galaxies: 9
RQ: 0; FR1: 0; FR2: 9; SB: 0; SF: 0.
Total number of simulated galaxies: 10
RQ: 0; FR1: 0; FR2: 10; SB: 0; SF: 0.
Total number of simulated galaxies: 11
RQ: 0; FR1: 0; FR2: 11; SB: 0; SF: 0.
Total number of simulated galaxies: 12
RQ: 0; FR1: 0; FR2: 12; SB: 0; SF: 0.
Total number of simulated galaxies: 13
RQ: 0; 

../utils/methods.py:62: RuntimeWarning: divide by zero encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)
../utils/methods.py:62: RuntimeWarning: invalid value encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)



RQ: 600; FR1: 285; FR2: 15; SB: 277; SF: 3600.
0.5377000000000001 3.19069 -2.5
Total number of simulated galaxies: 4778
RQ: 600; FR1: 285; FR2: 15; SB: 278; SF: 3600.
3.95265 3.34795 -2.5
Total number of simulated galaxies: 4779
RQ: 600; FR1: 285; FR2: 15; SB: 279; SF: 3600.
0.74796 0.68255 -2.5
Total number of simulated galaxies: 4780
RQ: 600; FR1: 285; FR2: 15; SB: 280; SF: 3600.
3.08975 1.19565 -2.5
Total number of simulated galaxies: 4781
RQ: 600; FR1: 285; FR2: 15; SB: 281; SF: 3600.
1.26159 3.23746 -2.5
Total number of simulated galaxies: 4782
RQ: 600; FR1: 285; FR2: 15; SB: 282; SF: 3600.
1.00116 3.26344 -2.5
Total number of simulated galaxies: 4783
RQ: 600; FR1: 285; FR2: 15; SB: 283; SF: 3600.
4.61279 4.26014 -2.5
Total number of simulated galaxies: 4784
RQ: 600; FR1: 285; FR2: 15; SB: 284; SF: 3600.
3.49894 4.64403 -2.5
Total number of simulated galaxies: 4785
RQ: 600; FR1: 285; FR2: 15; SB: 285; SF: 3600.
1.08415 0.9192200000000001 -2.5
Total number of simulated galaxies: 4

../utils/methods.py:62: RuntimeWarning: divide by zero encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)
../utils/methods.py:62: RuntimeWarning: invalid value encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)



RQ: 600; FR1: 285; FR2: 15; SB: 543; SF: 3600.
0.42015 3.81061 -2.5
Total number of simulated galaxies: 5044
RQ: 600; FR1: 285; FR2: 15; SB: 544; SF: 3600.
2.95513 4.15815 -2.5
Total number of simulated galaxies: 5045
RQ: 600; FR1: 285; FR2: 15; SB: 545; SF: 3600.
2.12588 0.7175900000000002 -2.5
Total number of simulated galaxies: 5046
RQ: 600; FR1: 285; FR2: 15; SB: 546; SF: 3600.
1.77127 1.6057799999999998 -2.5
Total number of simulated galaxies: 5047
RQ: 600; FR1: 285; FR2: 15; SB: 547; SF: 3600.
0.9380999999999999 1.0332700000000001 -2.5
Total number of simulated galaxies: 5048
RQ: 600; FR1: 285; FR2: 15; SB: 548; SF: 3600.
0.9342700000000002 2.95892 -2.5
Total number of simulated galaxies: 5049
RQ: 600; FR1: 285; FR2: 15; SB: 549; SF: 3600.
3.00059 0.91734 -2.5
Total number of simulated galaxies: 5050
RQ: 600; FR1: 285; FR2: 15; SB: 550; SF: 3600.
0.8674299999999999 0.8750200000000001 -2.5
Total number of simulated galaxies: 5051
RQ: 600; FR1: 285; FR2: 15; SB: 551; SF: 3600.
2.7

../utils/methods.py:62: RuntimeWarning: divide by zero encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)
../utils/methods.py:62: RuntimeWarning: invalid value encountered in true_divide
  distances = ((x_rotation)**2)/(a**2)+((y_rotation)**2)/(b**2)


3.19334 1.17566 -2.5
Total number of simulated galaxies: 5323
RQ: 600; FR1: 285; FR2: 15; SB: 823; SF: 3600.
4.62801 1.05799 -2.5
Total number of simulated galaxies: 5324
RQ: 600; FR1: 285; FR2: 15; SB: 824; SF: 3600.
4.524979999999999 2.74881 -2.5
Total number of simulated galaxies: 5325
RQ: 600; FR1: 285; FR2: 15; SB: 825; SF: 3600.
1.4768 1.0904 -2.5
Total number of simulated galaxies: 5326
RQ: 600; FR1: 285; FR2: 15; SB: 826; SF: 3600.
2.02374 0.7628900000000001 -2.5
Total number of simulated galaxies: 5327
RQ: 600; FR1: 285; FR2: 15; SB: 827; SF: 3600.
0.22153 0.99041 -2.5
Total number of simulated galaxies: 5328
RQ: 600; FR1: 285; FR2: 15; SB: 828; SF: 3600.
1.62995 1.76071 -2.5
Total number of simulated galaxies: 5329
RQ: 600; FR1: 285; FR2: 15; SB: 829; SF: 3600.
2.81541 2.0090500000000002 -2.5
Total number of simulated galaxies: 5330
RQ: 600; FR1: 285; FR2: 15; SB: 830; SF: 3600.
2.70087 3.97826 -2.5
Total number of simulated galaxies: 5331
RQ: 600; FR1: 285; FR2: 15; SB: 831;

In [10]:
len(Sim.skymodel)

6000

In [11]:
d = datetime.now()
dd = d.strftime('%m_%d_%Y_%H%M%S')

filename = 'skymodel_151_s' + str(len(Sim.skymodel)) + '_' + dd + '.pkl'

In [12]:
filename

'skymodel_151_s6000_08_28_2022_161321.pkl'

In [13]:
#Save skymodel
import timeit
tic=timeit.default_timer()
pickle_file = open(filename, 'wb')
pickle.dump(Sim.skymodel, pickle_file)
pickle_file.close()
toc=timeit.default_timer()
print("Dumping time:", toc - tic)

Dumping time: 0.02035935199998562


In [14]:
def pinfo():
    print(
    """
    Return pixel information of the simulated source.
    0:x, 1:y: x coordinate, y coordinate in a image coordinate system.
    2:i_151: Flux of the pixel at position (x, y) at 151 MHz.
    3:151: Simulation frequency
    4:redshift: Redshift of the galaxy
    5:int(agntype)/int(sftype): Galaxy type
        1: Radio quiet
        2: FR1
        3: FR2
        4: Star forming
        5: Star burst
    6:int(galaxy): Galaxy number
    7:i_151_tot: Total flux of the galaxy
    8:i_151_core: Core flux
    9:i_151_lobe1: Lobe flux
    10:i_151_lobe2: Lobe flux
    11:i_151_hotspot1: Hotspot flux
    12:i_151_hotspot2: Hotspot flux
    13:structure: Structure
    """)

def get_flux(model):
    return model[0][7]

## Convert Sim sky object to updated GMM skymodel

In [15]:
skymodel_151 = Sim.skymodel
PARAMS = Params
WDB = DB

In [16]:
skymodel_151

[array([[4.41000000e+02, 5.26300000e+03, 6.52228842e-04, 1.51000000e+02,
         1.44765800e+00, 3.00000000e+00, 5.99859070e+07, 3.63832011e-01,
         6.52228842e-04, 8.49180475e-02, 8.49180475e-02, 9.66718434e-02,
         9.66718434e-02, 1.00000000e+00],
        [4.41000000e+02, 5.26500000e+03, 9.66718434e-02, 1.51000000e+02,
         1.44765800e+00, 3.00000000e+00, 5.99859070e+07, 3.63832011e-01,
         6.52228842e-04, 8.49180475e-02, 8.49180475e-02, 9.66718434e-02,
         9.66718434e-02, 3.00000000e+00],
        [4.40000000e+02, 5.26100000e+03, 9.66718434e-02, 1.51000000e+02,
         1.44765800e+00, 3.00000000e+00, 5.99859070e+07, 3.63832011e-01,
         6.52228842e-04, 8.49180475e-02, 8.49180475e-02, 9.66718434e-02,
         9.66718434e-02, 3.00000000e+00],
        [4.41000000e+02, 5.26500000e+03, 8.49180475e-02, 1.51000000e+02,
         1.44765800e+00, 3.00000000e+00, 5.99859070e+07, 3.63832011e-01,
         6.52228842e-04, 8.49180475e-02, 8.49180475e-02, 9.66718434e-02

In [17]:
skymodel4000_151 = read_skymodel('skymodel_151_s4000_2022-08-15-v1.pkl')

In [18]:
skymodel_GMM_151 = []
GSM = GenSkyModel(PARAMS)
n = 0
for i in skymodel_151:
    galaxy = i[0, 6]
    gtype = i[0, 5]
    #print('galaxy:',galaxy)
    if gtype == 1:
        g = WDB.rq.loc[WDB.rq['galaxy'] == galaxy]
        mod = GSM.rq(g, model='GMM', Bmajor=1)
    if gtype == 2:
        g = WDB.fr1.loc[WDB.fr1['galaxy'] == galaxy]
        #print(g)
        mod = GSM.fr1(g, model='GMM', Bmajor=1)
    if gtype == 3:
        try:
            g = WDB.fr2.loc[WDB.fr2['galaxy'] == galaxy]
            #print(g)
            mod = GSM.fr2(g, model='GMM', Bmajor=1)
        except IndexError as error:
            print('Boundry issue need a fix')
            print('g:', g)
            print('i', i)
            print('n', n)
            print('End End End')
    if gtype == 4:
        g = WDB.sf.loc[WDB.sf['galaxy'] == galaxy]
        mod = GSM.sfsb(g, model='GMM', Bmajor=1)
    if gtype == 5:
        g = WDB.sb.loc[WDB.sb['galaxy'] == galaxy] 
        mod = GSM.sfsb(g, model='GMM', Bmajor=1)
    skymodel_GMM_151.append(mod) 
    n += 1
#     print(n)
#     print(mod['i_151_tot'], np.sum(mod['data'][:, 2]))

    if n % 100 == 0:
        print(n)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


In [19]:
GMMfilename = 'GMM_' + filename
save_skymodel(GMMfilename, skymodel_GMM_151)